# Word Error Rate
## GENFI (ES - EN)
Vamos a sacar este parametro en base a codigo que vamos constuyendo y sacando de internet.

Primero vamos a sacar a los achivos de sus carpetas de la task force para ordenar esto.

In [2]:
import os
import shutil

def copiar_txt_desde_carpetas(lista_carpetas, carpeta_destino="/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt"):
    # Crear la carpeta de destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    for carpeta in lista_carpetas:
        if os.path.exists(carpeta):  # Verificamos que exista la carpeta
            for archivo in os.listdir(carpeta):
                if archivo.endswith(".txt"):
                    ruta_origen = os.path.join(carpeta, archivo)
                    ruta_destino = os.path.join(carpeta_destino, archivo)

                    # Evitamos sobreescribir si hay archivos con el mismo nombre
                    contador = 1
                    nombre_archivo, extension = os.path.splitext(archivo)
                    while os.path.exists(ruta_destino):
                        ruta_destino = os.path.join(
                            carpeta_destino, f"{nombre_archivo}_{contador}{extension}")
                        contador += 1

                    shutil.copy2(ruta_origen, ruta_destino)
                    print(f"✅ Copiado: {ruta_origen} -> {ruta_destino}")
        else:
            print(f"⚠️ La carpeta {carpeta} no existe.")

# 👉 USO:
# Lista de carpetas que querés procesar
carpetas_a_buscar = [
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus",
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Miguel",
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Rafa",
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Vanesa"
]

copiar_txt_desde_carpetas(carpetas_a_buscar)


✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/01-167__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt/01-167__FREE__transcription.txt
✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/07-008__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt/07-008__FREE__transcription.txt
✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/01-126__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt/01-126__FREE__transcription.txt
✅ Copiado: /home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/Agus/07-011__FREE__transcription.txt -> /home/matias/Documents/UdeSA/CNC/diariziation_

Las carpetas quedan en:

- GROUND TRUTH: diariziation_error_rate/GENFI_Error_Rates/WER-ground-truth/all_txt
- PREDICTION: diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt

In [3]:
import os
import pandas as pd

# Rutas de las carpetas (modificá según corresponda)
carpeta_pred = "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-prediction/all_txt"
carpeta_gt = "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-ground-truth/all_txt"

# Inicializamos una lista donde guardaremos los registros
datos = []

# Recorremos todos los archivos de la carpeta de prediction
for archivo in os.listdir(carpeta_pred):
    if archivo.endswith(".txt"):
        id_archivo = os.path.splitext(archivo)[0]

        # Armamos las rutas de ambos archivos
        path_pred = os.path.join(carpeta_pred, archivo)
        path_gt = os.path.join(carpeta_gt, archivo)

        # Leemos prediction
        with open(path_pred, "r", encoding="utf-8") as f:
            prediction = f.read().strip()

        # Leemos ground_truth (si existe)
        if os.path.exists(path_gt):
            with open(path_gt, "r", encoding="utf-8") as f:
                ground_truth = f.read().strip()
        else:
            ground_truth = ""  # Si no existe, lo dejamos vacío

        # Guardamos los datos
        datos.append({
            "ID": id_archivo,
            "ground_truth": ground_truth,
            "prediction": prediction
        })

# Convertimos a DataFrame y exportamos a CSV
df = pd.DataFrame(datos)
df.to_csv("/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-comparacion.csv", index=False, encoding="utf-8")

print("✅ Archivo WER-comparacion.csv generado correctamente.")


✅ Archivo WER-comparacion.csv generado correctamente.


# WER Genfi

In [2]:
import os
from jiwer import wer
import pandas as pd


genfi_df = pd.read_csv("/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-comparacion.csv")


# calculate over all the dataset
wer_list = []
wer_dict = {}
for index, row in genfi_df.iterrows():
    try:
        reference = row["ground_truth"]
        hypothesis = row["prediction"]
        reference = reference.lower()
        hypothesis = hypothesis.lower()
        reference = reference.replace(".", "").replace(",", "").replace(";", "").replace(":", "").replace("!", "").replace("?", "").replace("¿", "").replace("¡", "")
        hypothesis = hypothesis.replace(".", "").replace(",", "").replace(";", "").replace(":", "").replace("!", "").replace("?", "")
        error = wer(reference, hypothesis)
        wer_list.append(error)
        wer_dict[row["ID"]] = error
    except AttributeError:
        print("ERROR: ", reference, hypothesis)
df = pd.DataFrame.from_dict(wer_dict, orient='index', columns=['WER'])
df.to_csv("/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WERs.csv", index=True, encoding="utf-8")
print("WER: ", sum(wer_list)/len(wer_list))


WER:  0.09100795227003505


## 1. Pipeline de Normalización de Texto

1. **Case folding**
   Convierte todo a minúsculas para evitar discrepancias (“Hola” vs “hola”).

2. **Eliminación o unificación de signos de puntuación**

   * Decide un conjunto fijo de símbolos a eliminar (.,;:¡!¿?–—)
   * O bien, si Whisper produce puntuación, inclúyela como “tokens” en la evaluación.

3. **Normalización de números**

   * **Opción A**: conviértelos a palabras (“123” → “ciento veintitrés” / “one hundred twenty three”).
   * **Opción B**: mantenlos como dígitos pero define reglas claras (p. ej. “1,234.56” uniforme a “1234.56”).
   * Utilizá librerías como `num2words` y parametrízalas por idioma.

4. **Manejo de acentos y caracteres especiales**

   * Puedes optar por **remover tildes** (`á` → `a`) o **mantenerlos**; lo importante es ser consistente.
   * Ejemplo: `unicodedata.normalize("NFD", text)` + filtro de `Mn`.

5. **Expansión/coerción de contracciones o abreviaturas**

   * Inglés: “I’m” → “i am”, “can’t” → “cannot”.
   * Español: “al” → “a el”, “del” → “de el” (si te interesa ser exhaustivo).

6. **Eliminación de “fillers” o ruidos**

   * Si tus textos de referencia no los contienen, mejor filtralos del output (`“uh”, “eh”, “[laugh]”`, etc.).

7. **Tokenización apropiada por idioma**

   * Usa tokenizadores ligeros (p. ej. `regex`, `split()`) o librerías dedicadas (SpaCy, NLTK) para español e inglés.

Con un preprocesamiento sólido, reduces sesgos entre idiomas y aseguras que la métrica mida **errores reales** de reconocimiento, no diferencias de estilo o formato.

---

## 2. Cálculo de Métricas Multilingüe

* **WER (*Word Error Rate*)**
  Evalúa inserciones, deleciones y sustituciones de tokens.
* **CER (*Character Error Rate*)**
  Útil para lenguajes con escritura aglutinante o scripts distintos.
* **Segmentación por idioma**

  * Divide tu conjunto en dos subconjuntos (español vs inglés) y calcula métricas por separado.
  * Permite detectar que quizás Whisper funciona mejor en uno que en otro.

---

## 3. Estimación de Incertidumbre y Reporte

1. **Intervalos de confianza**

   * Usa bootstrap sobre tus WERs por archivo para generar un intervalo (por ejemplo, 95 %).
   * Así no reportas solo el promedio sino el rango plausible.

2. **Desviación estándar / varianza**

   * Acompaña el WER promedio con su desviación, para mostrar dispersión.

3. **Matrices de confusión de tipo de error**

   * Por ejemplo, qué porcentaje de los errores fueron inserciones vs deleciones vs sustituciones.

4. **Desglose por longitud de segmento**

   * Segmentos cortos (≤ 3 palabras) suelen tener WER más volátil; muestra curvas de WER vs duración.

5. **Reporte tabulado y gráfico**

   * Haz tablas separadas por idioma y global.
   * Incluye gráficas de barras (WER por idioma), error bars, etc.

---

## 4. Cómo Reportar

* **Sección Metodología**: detalla paso a paso tu normalización (qué quitas, qué conservas).
* **Sección Resultados**: tablas de WER/CER por idioma, con sus intervalos de confianza.
* **Sección Discusión**: comenta si hay diferencias significativas entre español e inglés, cómo afectan los números, qué tipo de errores dominan.
* **Anexo**: ejemplos ilustrativos de transcripciones problemáticas (p. ej. “123” vs “ciento veintitrés”).

In [7]:
import os
import string
import unicodedata

import pandas as pd
import numpy as np
from jiwer import wer as jiwer_wer
from num2words import num2words

# 1. Función de normalización
def normalize(text, lang="es"):
    """
    - Pasa a minúsculas
    - Elimina tildes
    - Convierte dígitos a palabras (siempre que sean enteros)
    - Elimina puntuación
    """
    if not isinstance(text, str):
        return ""
    # case fold
    text = text.lower()
    # quitar tildes
    text = unicodedata.normalize("NFD", text)
    text = "".join(ch for ch in text if unicodedata.category(ch) != "Mn")
    # tokenizar simple
    tokens = text.split()
    # convertir números a palabras
    def num2word(tok):
        if tok.isdigit():
            try:
                return num2words(int(tok), lang=lang)
            except Exception:
                return tok
        return tok
    tokens = [num2word(tok) for tok in tokens]
    # eliminar puntuación
    table = str.maketrans("", "", string.punctuation + "¿¡")
    tokens = [tok.translate(table) for tok in tokens if tok.translate(table)]
    return " ".join(tokens)

# 2. Cargar datos
genfi_df = pd.read_csv(
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER-comparacion.csv"
)

# 3. Calcular WER por archivo
wer_scores = []
wer_dict = {}

for idx, row in genfi_df.iterrows():
    idx_id = row["ID"]
    if "15" in idx_id or "17" in idx_id:
        print(f"ID {idx_id} contiene 15 o 17, usando español")
        lang = "es"  # Cambiar a "es" si el ID contiene 15 o 17
    else:
        lang = "en"
    ref = normalize(row["ground_truth"], lang=lang)  # cambiar "es" o "en" según corresponda
    hyp = normalize(row["prediction"],  lang=lang)
    try:
        error = jiwer_wer(ref, hyp)
    except Exception as e:
        print(f"ERROR en {idx_id}: {e}")
        error = np.nan
    wer_scores.append(error)
    wer_dict[idx_id] = error

# 4. Guardar WERs individuales
df_wer = pd.DataFrame.from_dict(wer_dict, orient="index", columns=["WER"])
df_wer.index.name = "ID"
df_wer.to_csv(
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WERs.csv",
    encoding="utf-8",
)

# 5. Estadísticas globales
wer_array = np.array([x for x in wer_scores if not np.isnan(x)])
mean_wer = wer_array.mean()
std_wer  = wer_array.std(ddof=2)

# 6. Guardar estadísticas en un CSV aparte
stats_df = pd.DataFrame({
    "metric": ["WER_mean", "WER_std"],
    "value": [mean_wer, std_wer]
})
stats_df.to_csv(
    "/home/matias/Documents/UdeSA/CNC/diariziation_error_rate/GENFI_Error_Rates/WER_stats.csv",
    index=False,
    encoding="utf-8",
)

print(f"▶️ WER promedio: {mean_wer:.4f}")
print(f"▶️ Desviación estándar: {std_wer:.4f}")


ID 15-081 contiene 15 o 17, usando español
ID 17-029 contiene 15 o 17, usando español
ID 15-007 contiene 15 o 17, usando español
ID 15-087 contiene 15 o 17, usando español
ID 15-046 contiene 15 o 17, usando español
ID 15-025 contiene 15 o 17, usando español
ID 17-001 contiene 15 o 17, usando español
ID 15-035 contiene 15 o 17, usando español
ID 17-031 contiene 15 o 17, usando español
ID 15-028 contiene 15 o 17, usando español
ID 15-012 contiene 15 o 17, usando español
ID 15-078 contiene 15 o 17, usando español
ID 15-083 contiene 15 o 17, usando español
ID 17-020 contiene 15 o 17, usando español
ID 15-074 contiene 15 o 17, usando español
ID 15-006 contiene 15 o 17, usando español
ID 17-010 contiene 15 o 17, usando español
ID 15-069 contiene 15 o 17, usando español
ID 17-039 contiene 15 o 17, usando español
ID 15-021 contiene 15 o 17, usando español
ID 15-033 contiene 15 o 17, usando español
▶️ WER promedio: 0.0792
▶️ Desviación estándar: 0.1378
